In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [26]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import cv2

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


In [20]:
#This model is trained on kaggle soo the directories are of kaggle kernel and not of local machine
Batch_size = 32
image_size = tuple((256, 256))
directory= '../input/plant-village/'
width=256
height=256
depth=3

  ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [16]:
#This function converts image to array
def convert_image_to_array(image_dir):
    image = cv2.imread(image_dir)
    if image is not None :
        image = cv2.resize(image, image_size)   
        return img_to_array(image)

In [33]:
from os import listdir
import pickle

In [31]:
#Reading files nd storing them into image list and label list
image_list, label_list = [], []

print("[INFO] Loading images ...")
root_dir = listdir(directory)
    
for plant_folder in root_dir :
    plant_disease_folder_list = listdir(f"{directory}/{plant_folder}")
        
for plant_disease_folder in plant_disease_folder_list:
    print(f"[INFO] Processing {plant_disease_folder} ...")
    plant_disease_image_list = listdir(f"{directory}/{plant_folder}/{plant_disease_folder}/")
    for image in plant_disease_image_list[:200]:
        image_directory = f"{directory}/{plant_folder}/{plant_disease_folder}/{image}"
        if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
            image_list.append(convert_image_to_array(image_directory))
            label_list.append(plant_disease_folder)
print("[INFO] Image loading completed")  


[INFO] Loading images ...
[INFO] Processing Pepper__bell___Bacterial_spot ...
[INFO] Processing Pepper__bell___healthy ...
[INFO] Processing Potato___Late_blight ...
[INFO] Processing Tomato_Bacterial_spot ...
[INFO] Processing Tomato_healthy ...
[INFO] Processing Tomato_Leaf_Mold ...
[INFO] Processing Tomato_Septoria_leaf_spot ...
[INFO] Processing Potato___healthy ...
[INFO] Processing Tomato__Target_Spot ...
[INFO] Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
[INFO] Processing Potato___Early_blight ...
[INFO] Processing Tomato_Late_blight ...
[INFO] Processing Tomato__Tomato_mosaic_virus ...
[INFO] Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
[INFO] Processing Tomato_Early_blight ...
[INFO] Image loading completed


In [37]:
#Labelling our diseases as computer understands numbers and not named diseases
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
image_labels = lb.fit_transform(label_list)
pickle.dump(lb,open('label_for_disease.pkl', 'wb'))
n_classes = len(lb.classes_)

In [38]:
image_size = len(image_list)

In [39]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [40]:
#Splitting our data
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.3, random_state = 42) 

In [41]:
#Here i am using 300 samples per disease so this samples are used to generate more images with image data generator
#The images are flipped and rotated to generate new images
aug = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    horizontal_flip=True,vertical_flip=True 
    )

In [42]:
#Creating our model
model = Sequential()
inputShape = (height, width, depth)
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 85, 85, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 85, 85, 128)      

In [48]:
#we will use adam optimizer
optimizer = Adam(lr=0.001, decay=0.001 /20 )
model.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=["accuracy"])


In [49]:
#Training the model for 10 epochs
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=Batch_size),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // Batch_size,
    epochs=10, verbose=1
    )

Epoch 1/10
64/64 [==============================] - 29s 457ms/step - loss: 0.2380 - accuracy: 0.9336 - val_loss: 0.2357 - val_accuracy: 0.9326
Epoch 2/10
64/64 [==============================] - 28s 435ms/step - loss: 0.2266 - accuracy: 0.9337 - val_loss: 0.2174 - val_accuracy: 0.9344
Epoch 3/10
64/64 [==============================] - 28s 432ms/step - loss: 0.2058 - accuracy: 0.9353 - val_loss: 0.2109 - val_accuracy: 0.9301
Epoch 4/10
64/64 [==============================] - 27s 424ms/step - loss: 0.1929 - accuracy: 0.9359 - val_loss: 0.1740 - val_accuracy: 0.9394
Epoch 5/10
64/64 [==============================] - 28s 430ms/step - loss: 0.1832 - accuracy: 0.9375 - val_loss: 0.1755 - val_accuracy: 0.9388
Epoch 6/10
64/64 [==============================] - 27s 424ms/step - loss: 0.1716 - accuracy: 0.9400 - val_loss: 0.1842 - val_accuracy: 0.9361
Epoch 7/10
64/64 [==============================] - 27s 421ms/step - loss: 0.1646 - accuracy: 0.9413 - val_loss: 0.1610 - val_accuracy: 0.9420

In [50]:
#Here the accuracy of model is 93.51 which is quite good
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

885/885 [==============================] - 2s 2ms/step
Test Accuracy: 93.51412653923035


In [51]:
#Saving the trained model for further use
pickle.dump(model,open('cnn_model_plant_disease.pkl', 'wb'))

In [ ]:
#Loading our prtrained model

In [3]:
from os import listdir
listdir()

['.ipynb_checkpoints',
 'cnn_model_plant_disease.pkl',
 'Plant disease classification 1512.ipynb']

In [10]:
import pickle
import keras

Using TensorFlow backend.


In [11]:
model=pickle.load(open('cnn_model_plant_disease.pkl','rb'))

W1216 14:26:02.485614  4368 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1216 14:26:05.273987  4368 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1216 14:26:08.199764  4368 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 85, 85, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 85, 85, 128)      